In [3]:
import pandas as pd;

data = pd.read_csv('./historical_data/OANDA_USDJPY_5.csv');

data.columns = ['Time', 'Open', 'High', 'Low', 'Close'];
data['Time'] = pd.to_datetime(data['Time'], format='%Y-%m-%dT%H:%M:%S%z');
data = data.set_index('Time')

print(data);

                              Open     High      Low    Close
Time                                                         
2024-11-03 19:00:00-03:00  152.334  152.382  152.334  152.382
2024-11-03 19:05:00-03:00  152.370  152.391  152.330  152.389
2024-11-03 19:10:00-03:00  152.379  152.454  152.378  152.436
2024-11-03 19:15:00-03:00  152.448  152.544  152.436  152.542
2024-11-03 19:20:00-03:00  152.544  152.576  152.474  152.474
...                            ...      ...      ...      ...
2025-02-12 13:40:00-03:00  154.754  154.800  154.734  154.760
2025-02-12 13:45:00-03:00  154.759  154.760  154.694  154.722
2025-02-12 13:50:00-03:00  154.722  154.734  154.664  154.674
2025-02-12 13:55:00-03:00  154.674  154.675  154.583  154.628
2025-02-12 14:00:00-03:00  154.627  154.632  154.510  154.510

[20387 rows x 4 columns]


In [18]:
from backtesting import Backtest, Strategy
import talib
import numpy as np
import pandas as pd

class EMABasedStrategy(Strategy):
    # Input parameters
    pips_sl = 25         # Stop Loss in pips
    pips_tp = 25         # Take Profit in pips
    pip_value = 0.005 * 10  # Adjustment for Forex (e.g., EURUSD); change if needed
    ema_length = 54      # EMA period
    confirm_bars = 20    # Required consecutive bars above/below EMA for trend confirmation
    distance_forex = 0.1 # Band distance percentage for Forex

    # Compute stop-loss and take-profit distances (static, based on entry price)
    sl_distance = pips_sl * pip_value
    tp_distance = pips_tp * pip_value

    def init(self):
        # Calculate the 54-period EMA on the Close price.
        self.ema54 = self.I(talib.EMA, self.data.Close, self.ema_length)
        
        # Compute the price range over the last 'ema_length' bars:
        # highest high minus lowest low over the period.
        self.price_range = self.I(
            lambda high, low: talib.MAX(high, timeperiod=self.ema_length) -
                                talib.MIN(low, timeperiod=self.ema_length),
            self.data.High, self.data.Low
        )
        
        # Calculate the distance for the upper and lower bands.
        self.distance_bands = self.price_range * self.distance_forex
        
        # Define the upper and lower bands around the EMA.
        self.upper_line = self.ema54 + self.distance_bands
        self.lower_line = self.ema54 - self.distance_bands
        
        # Initialize counters to count consecutive bars above or below the EMA.
        self.count_above = 0
        self.count_below = 0

    def next(self):
        price = self.data.Close[-1]
        ema = self.ema54[-1]
        upper = self.upper_line[-1]
        lower = self.lower_line[-1]

        # Update trend counters: if the current close is above EMA, increment; else, reset.
        if price > ema:
            self.count_above += 1
        else:
            self.count_above = 0

        # Similarly for bars below the EMA.
        if price < ema:
            self.count_below += 1
        else:
            self.count_below = 0

        # Check for trend confirmation.
        is_confirmed_uptrend = self.count_above >= self.confirm_bars
        is_confirmed_downtrend = self.count_below >= self.confirm_bars

        # Only enter a new trade if there is no open position.
        if not self.position:
            # Long entry condition:
            #   - Uptrend confirmed,
            #   - Price is at or above the EMA and below (or equal to) the upper band.
            if is_confirmed_uptrend and (price >= ema) and (price <= upper):
                # Place a long order with stop loss and take profit based on the entry price.
                self.buy(sl=price - self.sl_distance, tp=price + self.tp_distance)
            # Short entry condition:
            #   - Downtrend confirmed,
            #   - Price is at or below the EMA and above (or equal to) the lower band.
            elif is_confirmed_downtrend and (price <= ema) and (price >= lower):
                self.sell(sl=price + self.sl_distance, tp=price - self.tp_distance)
        # Note: With the sl and tp parameters attached to the entry orders, Backtesting.py
        # will automatically exit the position when price reaches these levels.
        
# Create and run the backtest:
bt = Backtest(data, EMABasedStrategy, cash=10_000, commission=0, exclusive_orders=True)
stats = bt.run()
print(stats)
bt.plot()


Start                     2024-11-03 19:00...
End                       2025-02-12 14:00...
Duration                    100 days 19:00:00
Exposure Time [%]                    92.50503
Equity Final [$]                    10320.874
Equity Peak [$]                     10320.874
Return [%]                            3.20874
Buy & Hold Return [%]                 1.72895
Return (Ann.) [%]                     9.46594
Volatility (Ann.) [%]                 8.60849
CAGR [%]                              8.21663
Sharpe Ratio                          1.09961
Sortino Ratio                         1.80276
Calmar Ratio                          2.48691
Max. Drawdown [%]                    -3.80631
Avg. Drawdown [%]                    -0.27382
Max. Drawdown Duration       43 days 10:10:00
Avg. Drawdown Duration        2 days 01:51:00
# Trades                                   34
Win Rate [%]                         55.88235
Best Trade [%]                        0.83237
Worst Trade [%]                   

/opt/conda/lib/python3.10/site-packages/backtesting/_plotting.py:125: UserWarning: Data contains too many candlesticks to plot; downsampling to '15min'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/opt/conda/lib/python3.10/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p2051', ...)

In [17]:
print(stats)


Start                     2024-11-03 19:00...
End                       2025-02-12 14:00...
Duration                    100 days 19:00:00
Exposure Time [%]                    92.50503
Equity Final [$]                    10320.874
Equity Peak [$]                     10320.874
Return [%]                            3.20874
Buy & Hold Return [%]                 1.72895
Return (Ann.) [%]                     9.46594
Volatility (Ann.) [%]                 8.60849
CAGR [%]                              8.21663
Sharpe Ratio                          1.09961
Sortino Ratio                         1.80276
Calmar Ratio                          2.48691
Max. Drawdown [%]                    -3.80631
Avg. Drawdown [%]                    -0.27382
Max. Drawdown Duration       43 days 10:10:00
Avg. Drawdown Duration        2 days 01:51:00
# Trades                                   34
Win Rate [%]                         55.88235
Best Trade [%]                        0.83237
Worst Trade [%]                   